In [ ]:
import os
import numpy as np
import spectral.io.envi as envi
import matplotlib.pyplot as plt
from hsi_utils import *

from sklearn.decomposition import PCA

In [ ]:
#if using a jupyter notbook need a graphical backend TkaAgg or Qt5Agg if using spyder type %matplotlib auto in console
import matplotlib
matplotlib.use('TkAgg') 

In [ ]:
# Define the path to the main data folder: code will iterate trough relvant files
main_data_folder = './data/img_test'    

#check if data path exists !
print(os.path.isdir(main_data_folder))

# Initialize the HSI dataset and define file extension: contains all paths of hdr and data files
dataset =HsiDataset(main_data_folder,data_ext='hyspex') #raw files from the camera have '.hyspex' extension , corrected files have '.ref' extension
nb_images = len(dataset)
print(f"dataset contains {nb_images} image(s)")

In [ ]:
HSIreader = HsiReader(dataset)

In [ ]:
# Loop through each hyperspectral image in the dataset
# for idx in range(len(dataset)):
#     HSIreader.read_image(idx)

#choose an image to process e.g. first img idx=0
idx=0   
HSIreader.read_image(idx)
print(f"read image{HSIreader.current_name}")

metadata = HSIreader.current_metadata
print(metadata)

#define wavelenrghts (for plots mostly)
wv =HSIreader.get_wavelength()
wv = [int(l) for l in wv]

# Get the pseudo rgb imahe from hyperspectral data and plot image
pseudo_rgb= HSIreader.get_rgb()
# plt.figure()
# plt.imshow(pseudo_rgb)
# plt.axis('off')
# plt.show()

In [ ]:
#check spectral data by manually selecting pixel-spectrum samples and plot
HSIreader.get_spectrum();

In [ ]:
# Extract spectral sample from HSI for analysis

#read HSI specs without laoding it in memory!
hsi=HSIreader.current_image
n_rows, n_cols, n_channels =hsi.shape

n_samples =1000
# draw random samples and load them as a data array
x_idx = np.random.randint(0, n_cols, size=n_samples)
y_idx = np.random.randint(0, n_rows, size=n_samples)
spectral_samples = np.zeros((n_samples, n_channels), dtype=hsi.dtype)
coords = list(zip(y_idx, x_idx))
spectral_samples = np.array(HSIreader.extract_pixels(coords))

In [ ]:
plt.figure()
for i in range(n_samples):
    plt.plot(wv, spectral_samples[i, :], label=f'Sample {i+1}' if i < 5 else "", alpha=0.6)
plt.xlabel("Wavelength")
plt.ylabel("Absorbance")
plt.title("Spectral samples")
plt.show()


In [ ]:
#perform PCA
#In PCA space data X= TP' i.e. factorial decomposition into loadings (vectorial subspaces) and scores (orthogonal distances to loadings)

nb_pca_comp =3
pca = PCA(n_components=nb_pca_comp)
pca_scores = pca.fit_transform(spectral_samples)
pca_loadings =pca.components_.T*np.sqrt(pca.explained_variance_) 
      
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.figure()
for i in range(np.shape(pca_loadings)[1]):
    lab= 'PC'+str(i+1)
    plt.plot(wv,pca_loadings[:,i],default_colors[i], label=lab)
plt.xlabel("Wavelength (nm)")
plt.ylabel("Absorbance")  
plt.grid()
plt.title('Principal components')  
plt.show()

#Or plot one by one to avoid Variance scaling
# default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# for i in range(np.shape(pca_loadings)[1]):
#     plt.figure()
#     plt.plot(wv,pca_loadings[:,i],default_colors[i])
#     plt.xlabel("Wavelength (nm)")
#     plt.ylabel("Absorbance")  
#     lab= 'PC'+str(i+1)
#     plt.title(lab) 
#     plt.grid()  
# plt.show()